In [ ]:
import os

import pandas as pd

import pareto

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

In [ ]:
dynamic_path = norm(join(cwd, '../dynamic/runs/dynamic_stats.csv'))
static_path = norm(join(cwd, '../static/runs/static_stats.csv'))
mlaa_path = norm(join(cwd, '../MLAA-Bernier/runs/MLAA_stats.csv'))
liu_path = norm(join(cwd, '../Hierarchical-Liu/runs/Hierarchical_stats.csv'))
cenk_path = norm(join(cwd, '../Yavuzturk/runs/Yavuzturk_stats.csv'))

In [ ]:
df_d = pd.read_csv(dynamic_path, index_col=[0])
df_s = pd.read_csv(static_path, index_col=[0])
df_m = pd.read_csv(mlaa_path, index_col=[0])
df_l = pd.read_csv(liu_path, index_col=[0])
df_c = pd.read_csv(cenk_path, index_col=[0])

In [ ]:
df_c.head(2)

In [ ]:
def plot_all_methods_runtimefrac_vs_rmse(dfs, names, load, year):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    
    for idx, df in enumerate(dfs):
        mask = (df['load'] == load) & (df['sim time'] == year)
        x = df.loc[mask]['rmse']
        y = df.loc[mask]['run time fraction']
        ax.scatter(x, y, label=names[idx])
    
    plt.xlabel('RMSE MFT [C]')
    plt.ylabel('Runtime Fraction')
    plt.title('{} {}'.format(load.title(), year))
    
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plot_all_methods_runtimefrac_vs_rmse([df_d, df_s, df_m, df_l, df_c], 
                                     ['Dynamic', 'Static', 'Bernier', 'Liu', 'Yavuzturk'],
                                    'balanced',
                                    1)

In [ ]:
plot_all_methods_runtimefrac_vs_rmse([df_d, df_s, df_m, df_l, df_c], 
                                     ['Dynamic', 'Static', 'Bernier', 'Liu', 'Yavuzturk'],
                                    'imbalanced',
                                    1)

In [ ]:
plot_all_methods_runtimefrac_vs_rmse([df_d, df_s, df_m, df_l, df_c], 
                                     ['Dynamic', 'Static', 'Bernier', 'Liu', 'Yavuzturk'],
                                    'balanced',
                                    5)

In [ ]:
plot_all_methods_runtimefrac_vs_rmse([df_d, df_s, df_m, df_l, df_c], 
                                     ['Dynamic', 'Static', 'Bernier', 'Liu', 'Yavuzturk'],
                                    'imbalanced',
                                    5)

In [ ]:
m_b1 = (df['load'] == 'balanced') & (df['sim time'] == 1)
m_b2 = (df['load'] == 'balanced') & (df['sim time'] == 2)
m_b3 = (df['load'] == 'balanced') & (df['sim time'] == 3)
m_b4 = (df['load'] == 'balanced') & (df['sim time'] == 4)
m_b5 = (df['load'] == 'balanced') & (df['sim time'] == 5)
m_b6 = (df['load'] == 'balanced') & (df['sim time'] == 6)

m_i1 = (df['load'] == 'imbalanced') & (df['sim time'] == 1)
m_i2 = (df['load'] == 'imbalanced') & (df['sim time'] == 2)
m_i3 = (df['load'] == 'imbalanced') & (df['sim time'] == 3)
m_i4 = (df['load'] == 'imbalanced') & (df['sim time'] == 4)
m_i5 = (df['load'] == 'imbalanced') & (df['sim time'] == 5)
m_i6 = (df['load'] == 'imbalanced') & (df['sim time'] == 6)

In [ ]:
exp_rates = ['1.25', '1.50', '1.62', '1.75', '2.00', '2.25', '2.50', '2.75', '3.00']

markers = ['o', 'v', '^', '<', '>', 'p', 'P', 's', 'h', '+', 'X', 'x', 'h', 'H', 'D', 'd']

In [ ]:
def make_fig(df_in, mask_series, mask_col_name, title=None):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    for idx, mask in enumerate(reversed(mask_series)):
        s = df_in[mask_col_name] == float(mask)

        x = df_in.loc[s]['rmse']
        y = df_in.loc[s]['run time']

        m = markers[idx]
        ax.scatter(x, y, marker=m, label=mask, s=60)

    if title:
        plt.title(title)
        
    plt.legend()
    plt.show()

In [ ]:
make_fig(df.loc[m_b1], exp_rates, 'exp_rate', '1-year Balanced')
make_fig(df.loc[m_b2], exp_rates, 'exp_rate', '2-year Balanced')
make_fig(df.loc[m_b3], exp_rates, 'exp_rate', '3-year Balanced')
make_fig(df.loc[m_b4], exp_rates, 'exp_rate', '4-year Balanced')
make_fig(df.loc[m_b5], exp_rates, 'exp_rate', '5-year Balanced')
make_fig(df.loc[m_b6], exp_rates, 'exp_rate', '6-year Balanced')

In [ ]:
make_fig(df.loc[m_i1], exp_rates, 'exp_rate', '1-year Imbalanced')
make_fig(df.loc[m_i2], exp_rates, 'exp_rate', '2-year Imbalanced')
make_fig(df.loc[m_i3], exp_rates, 'exp_rate', '3-year Imbalanced')
make_fig(df.loc[m_i4], exp_rates, 'exp_rate', '4-year Imbalanced')
make_fig(df.loc[m_i5], exp_rates, 'exp_rate', '5-year Imbalanced')
make_fig(df.loc[m_i6], exp_rates, 'exp_rate', '6-year Imbalanced')

In [ ]:
def define_pareto(df_in):
    df = pd.DataFrame.from_records(pareto.eps_sort([list(df_in.itertuples(False))], [4, 5]), columns=list(df_in.columns.values))
    df.sort_values(by=['rmse'], inplace=True)
    return df

In [ ]:
pareto_b1 = define_pareto(df.loc[m_b1])
pareto_b2 = define_pareto(df.loc[m_b2])
pareto_b3 = define_pareto(df.loc[m_b3])
pareto_b4 = define_pareto(df.loc[m_b4])
pareto_b5 = define_pareto(df.loc[m_b5])
pareto_b6 = define_pareto(df.loc[m_b6])

In [ ]:
pareto_i1 = define_pareto(df.loc[m_i1])
pareto_i2 = define_pareto(df.loc[m_i2])
pareto_i3 = define_pareto(df.loc[m_i3])
pareto_i4 = define_pareto(df.loc[m_i4])
pareto_i5 = define_pareto(df.loc[m_i5])
pareto_i6 = define_pareto(df.loc[m_i6])

In [ ]:
make_fig(pareto_b1, exp_rates, 'exp_rate', '1-year Balanced')
make_fig(pareto_b2, exp_rates, 'exp_rate', '2-year Balanced')
make_fig(pareto_b3, exp_rates, 'exp_rate', '3-year Balanced')
make_fig(pareto_b4, exp_rates, 'exp_rate', '4-year Balanced')
make_fig(pareto_b5, exp_rates, 'exp_rate', '5-year Balanced')
make_fig(pareto_b6, exp_rates, 'exp_rate', '6-year Balanced')

In [ ]:
make_fig(pareto_i1, exp_rates, 'exp_rate', '1-year Imbalanced')
make_fig(pareto_i2, exp_rates, 'exp_rate', '2-year Imbalanced')
make_fig(pareto_i3, exp_rates, 'exp_rate', '3-year Imbalanced')
make_fig(pareto_i4, exp_rates, 'exp_rate', '4-year Imbalanced')
make_fig(pareto_i5, exp_rates, 'exp_rate', '5-year Imbalanced')
make_fig(pareto_i6, exp_rates, 'exp_rate', '6-year Imbalanced')

In [ ]:
def make_fig_with_annotation(df_in, mask_series, mask_col_name, annotate_col_name, title=None):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    for idx, mask in enumerate(reversed(mask_series)):
        s = df_in[mask_col_name] == float(mask)

        x = df_in.loc[s]['rmse']
        y = df_in.loc[s]['run time']

        m = markers[idx]
        ax.scatter(x, y, marker=m, label=mask, s=60)
              
        for i, txt in enumerate(df_in.loc[s][annotate_col_name].values):
            ax.annotate(txt, (x.values[i], y.values[i]))

    if title:
        plt.title(title)
        
    plt.legend()
    plt.show()

In [ ]:
exp_rate_mask = df['exp_rate'] == 1.75

In [ ]:
start_widths = range(1, 6)

In [ ]:
make_fig_with_annotation(df.loc[m_b1 & exp_rate_mask], start_widths, 'start width', 'end width', '1-year Balanced')
make_fig_with_annotation(df.loc[m_b2 & exp_rate_mask], start_widths, 'start width', 'end width', '2-year Balanced')
make_fig_with_annotation(df.loc[m_b3 & exp_rate_mask], start_widths, 'start width', 'end width', '3-year Balanced')
make_fig_with_annotation(df.loc[m_b4 & exp_rate_mask], start_widths, 'start width', 'end width', '4-year Balanced')
make_fig_with_annotation(df.loc[m_b5 & exp_rate_mask], start_widths, 'start width', 'end width', '5-year Balanced')
make_fig_with_annotation(df.loc[m_b6 & exp_rate_mask], start_widths, 'start width', 'end width', '6-year Balanced')

In [ ]:
make_fig_with_annotation(df.loc[m_i1 & exp_rate_mask], start_widths, 'start width', 'end width', '1-year Imbalanced')
make_fig_with_annotation(df.loc[m_i2 & exp_rate_mask], start_widths, 'start width', 'end width', '2-year Imbalanced')
make_fig_with_annotation(df.loc[m_i3 & exp_rate_mask], start_widths, 'start width', 'end width', '3-year Imbalanced')
make_fig_with_annotation(df.loc[m_i4 & exp_rate_mask], start_widths, 'start width', 'end width', '4-year Imbalanced')
make_fig_with_annotation(df.loc[m_i5 & exp_rate_mask], start_widths, 'start width', 'end width', '5-year Imbalanced')
make_fig_with_annotation(df.loc[m_i6 & exp_rate_mask], start_widths, 'start width', 'end width', '6-year Imbalanced')

In [ ]:
def make_some_plot(*args):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    
    args = args[0]
    
    for s in args:
        print(args)
        try:
            ax.plot(s['x'], s['y'], label=s['label'])
        except KeyError:
                ax.plot(s['x'], s['y'])
        
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
make_some_plot([a, b, c])